In [ ]:
# Library imports
import numpy as np

In [ ]:
def mse(data, betas):
    rows, cols = data.shape
    loss = 0
    for i in range(rows):
        u = betas[0] + np.dot(betas[1:], data[i][:cols - 1])
        loss += pow(data[i][cols - 1] - u, 2)
    loss *= 1 / rows
    return loss

def iterate(data, betas, epochs, learning_rate):
    rows, cols = data.shape
    metrics = {

    }
    for k in range(epochs):
        dbetas = np.zeros(len(betas))
        for i in range(rows):
            temp = 0
            dbetas[0] += temp
            
        dbetas *= -2 / rows
        betas -= dbetas * learning_rate
    return metrics